In [1]:
# import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import datetime as dt
from matplotlib  import rcParams

%matplotlib inline 

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [50]:
%matplotlib inline
# rcParams["figure.figsize"] = 16,6
c = '#386B7F'

In [2]:
# upload the bank term deposit data set
data = pd.read_csv("https://github.com/muluwork-shegaw/10Academy-week6/blob/master/data/bank-additional-full.csv?raw=true",error_bad_lines=False,sep=';')
# data_2 = pd.read_excel("bank-additional/bank-additional-full-1.csv", low_memory=False)

# Task 1

#### Summery about the data

      1) bank-additional-full.csv with all examples, ordered by date (from May 2008 to November 2010).
      2) bank-additional.csv with 10% of the examples (4119), randomly selected from bank-additional-full.csv.
   The smallest dataset is provided to test more computationally demanding machine learning algorithms (e.g., SVM).

   The binary classification goal is to predict if the client will subscribe a bank term deposit (variable y).

1. Number of Instances: 41188 for bank-additional-full.csv

2. Number of Attributes: 20 + output attribute.
3. **Missing Attribute Values: There are several missing values in some categorical attributes, all coded with the "unknown" label. These missing values can be treated as a possible class label or using deletion or imputation techniques.** 
4. Attribute information:


   1. **bank client data:**
   
   1- age (numeric)
   
   2 - job : type of job (categorical: "admin.","blue-collar","entrepreneur","housemaid","management","retired","self-employed","services","student","technician","unemployed","unknown")
   
   3 - marital : marital status (categorical: "divorced","married","single","unknown"; note: "divorced" means divorced or widowed)
   4 - education (categorical: "basic.4y","basic.6y","basic.9y","high.school","illiterate","professional.course","university.degree","unknown")
   
   5 - default: has credit in default? (categorical: "no","yes","unknown")
   
   6 - housing: has housing loan? (categorical: "no","yes","unknown")
   
   7 - loan: has personal loan? (categorical: "no","yes","unknown")
   2. **related with the last contact of the current campaign:**
   
   8 - contact: contact communication type (categorical: "cellular","telephone") 
   
   9 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")
   
   10 - day_of_week: last contact day of the week (categorical: "mon","tue","wed","thu","fri")
   
   11 - duration: last contact duration, in seconds (numeric). Important note:  this attribute highly affects the output target (e.g., if duration=0 then y="no"). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
   3. **other attributes:**
   
   12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
  
   13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
   
   14 - previous: number of contacts performed before this campaign and for this client (numeric)
   
   15 - poutcome: outcome of the previous marketing campaign (categorical: "failure","nonexistent","success")
   4. **social and economic context attributes**
   
   16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
   
   17 - cons.price.idx: consumer price index - monthly indicator (numeric)    
   
   18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)  
   
   19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
   
   20 - nr.employed: number of employees - quarterly indicator (numeric)

   5. Output variable (desired target):
   
   21 - y - has the client subscribed a term deposit? (binary: "yes","no")




### Data Glance

In [ ]:
#glance the data
print("shape of the  store data: ",data.shape)
print("\n unique no of each variable\n")
print(data.nunique())
print("\n--------data summery------------\n")
print(data.info())
data.sample(10)

### Univariate Analysis

In [4]:
cat_col = data.select_dtypes(exclude=['int64', 'float64']).columns #catagorical columns
non_cat_col = data.select_dtypes(include=['int64', 'float64']).columns # non catagorical variable


In [ ]:
for  var in non_cat_col:
    g = sns.FacetGrid(data)
    g.map(sns.distplot, var,hist=True,kde=False)
    plt.show()

In [6]:

def percentage_plot(data,col = cat_col):
    total = float(len(data))
    if type(col)==str:
        plot = sns.countplot(col, data=data, order = data[col].value_counts().index)
        for p in plot.patches:
            percentage = '{:.1f}%'.format(100 * p.get_height()/total)
            x = p.get_x() + p.get_width()
            y = p.get_height()
            plot.annotate(percentage, (x, y),ha='center')

    else :
        fig, axs = plt.subplots(nrows=int(len(col)/2)+1, ncols=2, figsize=(20,40))
        for i, var in enumerate(col):
            row = i//2
            pos = i % 2
            plot = sns.countplot(var, data=data, ax=axs[row][pos],order = data[var].value_counts().index)
            for p in plot.patches:
                percentage = '{:.1f}%'.format(100 * p.get_height()/total)
                x = p.get_x() + p.get_width()
                y = p.get_height()
                plot.annotate(percentage, (x, y),ha='center')



In [ ]:
percentage_plot(data)



*- when we see the day_of_week ,the have equa1 probable pecentage, which means the days has no that much effect.*

**What is Imbalanced Data?**

Imbalanced data typically refers to a problem with classification problems where the classes are not represented equally.

For example, we may have a 2-class (binary) classification problem with 100 instances (rows). A total of 80 instances are labeled with Class-1 and the remaining 20 instances are labeled with Class-2.

This is an imbalanced dataset and the ratio of Class-1 to Class-2 instances is 80:20 or more concisely 4:1.

we can have a class imbalance problem on two-class classification problems as well as multi-class classification problems. Most techniques can be used on either.


**NB: As shown in figure above our target variable "y" 88.7 :11.3, there is class imbalance** 

soln:

1.     rater than using accuracy use other metrics like confusion matrix,recall etc.
2. Over-Sampling the minority class(yes class)
3. modeling using XGBoost is better for the data given

### Bivariate

In [ ]:
for  var in cat_col:
    total_1 = float(len(data[data["y"]=="no"]))
    total_2 = float(len(data[data["y"]=="yes"]))
    total =[total_1,total_2]
    i=0
    

    g = sns.catplot(x=var,kind='count', hue="y",
                col="y", data=data,order = data[var].value_counts().index)
    for ax in g.axes.ravel():
            for p in ax.patches:
                percentage = '{:.1f}%'.format(100 * p.get_height()/total[i])
                x = p.get_x() + p.get_width()
                y = p.get_height()
                ax.annotate(percentage, (x, y),ha='center')
            i =i+1

In [ ]:

import warnings
warnings.filterwarnings("ignore")
for  var in non_cat_col:
    g = sns.FacetGrid(data,height=5)
    g.map(sns.boxplot,y=data[var], x=data["y"],hue=data["y"],color="orange",labels=[var])
    g.add_legend()
    plt.show()    
    print("                    {}\n".format(var))

   

In [10]:
def corr(data,col = None):
    corr_matrix = data[col].corr()

        
    mask = np.zeros_like(corr_matrix, dtype = np.bool)
    mask[np.triu_indices_from(mask)] = True
    f, ax = plt.subplots(figsize = (11, 9))

    sns.heatmap(corr_matrix,annot=True,mask=mask, linewidths=.3,ax = ax)
    plt.show()
    return corr_matrix

In [ ]:
corr_matrix=corr(data,non_cat_col)


In [ ]:
data[non_cat_col].plot(kind = 'box',showfliers = True,color = "orange") #with outliers


In [ ]:
data[non_cat_col].plot(kind = 'box',showfliers = False)#without outliers


# Preprocessing

Since our dataset has many variables(20 features with out the target variable) and unknow value , we should carefull when we deal with outliers,missing value and dimentionality reduction since there is a class imbalance between "yes" and "no" in [y] variable as shown in figure above, we may biase  our model unless we consider such things.

In [28]:

from sklearn.preprocessing import OneHotEncoder,LabelEncoder

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
# from keras.models import Model
# from Keras.layers  import Input,Dense
# from keras import regularizers
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

import time
time_start = time.time()


class PreProcessor:

    def __init__(self,data,target=None,drop_col=None,outlier_col=None,reduction_model=None,dim=None):
        self.data = data
        self.orig_data = data
        self.target = target
        self.drop_col =drop_col
        self.outlier_col =outlier_col
        
        self.reduction_model = reduction_model
        self.dim = dim
        
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.y_pred = None

        
        

    '''
       Missing Value Ratio
       calculate the missing value ration
       and drop that variable if it's missing value ratio 
       is greater than 20% and otherwise impute or drop that
       value

    ''' 

    def treat_missing(self,data):#feature selection based on missing value ratio
        cat_col = self.data.select_dtypes(exclude=['int64', 'float64']).columns #catagorical columns
        non_cat_col = self.data.select_dtypes(include=['int64', 'float64']).columns # non catagorical variable
    
        data[cat_col] = data[cat_col].replace(['unknown', 0], np.nan)
        missing_value = data.isnull().sum()/len(data)*100
        missing_var = []
        for i,col in enumerate(self.data.columns):
            if missing_value[i]>=20:   #setting the threshold as 20%
                missing_var.append(col)
                data.drop(col, axis=1,inplace=True)
        print("\n {} variables have dropped based on missing value ratio".format(missing_var))
          
        '''
            since the rest variable's missing value is 
            smaller we can impute them
        '''
        print("\n {} missing values have been dropped".format(data.isnull().sum().sum()))
        data=data.fillna(data.mode().iloc[0])
        
        self.data = data
        return data
    '''
        we may need to drop a columns which have a less variance and
        more correlated variables, since if they are more correlated 
        they have same impact in indecating something and if they have least
        variance which means they are almost constant values.
    '''
    def drope(self,data,col=None): # which is ongoing function,not implemented in optimized way yet.
        data.drop(col,axis =1,inplace=True)
        self.cat_col = data.select_dtypes(exclude=['int64', 'float64']).columns #catagorical columns
        self.non_cat = data.select_dtypes(include=['int64', 'float64']).columns #catagorical columns
        print("\n {} variable has been dropped based on high correlation and less variance analysis".format([col]))
        self.data = data
        return data

    def deal_outliers(self,data,columns):#handle outliers based on interquartile range(iqr)
        orig_data = data

        print("\n before dealing with the ouliers the shape of the data",
              data.shape)
        for col in columns:
            q1 = data[col].quantile(0.25) # quartile 1
            q3 = data[col].quantile(0.75) # quartile 4
            iqr = q3 - q1
            lower_bound = q1 -(1.5 * iqr) #lower whishker
            upper_bound = q3 +(1.5 * iqr) #upper whishker
            data = data[data[col]>lower_bound] #remove which are greater than upper whishker
            data = data[data[col]<upper_bound] #remove which are less than lower whishker

        print("after removing the outliers",data.shape)
        
        self.data = data
        self.orig_data = orig_data
        
        print("{}  data has been removed based on outlier analysis".format(len(self.orig_data)-len(self.data)))
        return data # return data with outlier and without outlier
    
    def encoder(self,data):# encode the catagorical value
        label_encoder = LabelEncoder()
        binary_col = []
        for col in (data.columns):
            if data[col].nunique() == 2:
                binary_col.append(col)
                data[col] = label_encoder.fit_transform(data[col])# encode binary variables
        print("\n{} variables have been encoded based on label encoding the rest encoded as dummy variable".format(binary_col))
        
        data = pd.get_dummies(data) #encode not binary variables
        self.data = data
        return data
    
    def scaler(self,data, scaler= MinMaxScaler()): # not implmented in optimized way yet
        data[:] = scaler.fit_transform(data[:]) #scaleing the given data
        self.data = data
        print("\n---- Scaling the data based on",scaler)
        return data
    
    def reduce_dimension(self,data,reduction_model = 'tsne',dim=3): # reduce the dimentions of a given data
        target = self.target
        data_1 = data
        d = data_1.drop(target,axis=1)
        
        print("\nThe data is reduced based on {0} algorithm to {1} dimentions".format(reduction_model,dim+1))
        print("Executing the reduction starts and wait ......")
        
        if reduction_model =='pca': # using PCA
            pca = PCA(n_components=dim)
            reduced_df = pd.DataFrame(pca.fit_transform(d), # reduce the dimention and convert to data frame
               columns=[f'pca {i}'  for i in range(1,dim+1)]) 
            print ('PCA done! Time elapsed: {} seconds'.format(time.time()-time_start))
            
        elif reduction_model =='tsne':
            tsne = TSNE(n_components = dim, n_iter = 300)
            reduced_df =pd.DataFrame(tsne.fit_transform(d),
                        columns=[f'tsne {i}'  for i in range(1,dim+1)])
            print ('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

    #     else:
    #         input_dim = d.shape[1]
    #         input_layer = Input(shape = (input_dim,))
    #         encoder_layer_1 = Dense(40,activation='tanh',
    #                         activity_regularizer=regularizers.l1(10e-5)(input))(input_layer)
    #         encoder_layer_2 = Dense(30,activation='tanh')(encoder_layer_1)
    #         encoder_layer_3 = Dense(encoding_dim,activation='tanh')(encoder_layer_2)
    #         #create encoder model
    #         encoder = Model(inputs = input_layer, outputs = encoder_layer_3)
    #         reduced_df= pd.DataFrame(encoder.predict(d))
                   
    # reset the index to get clean data(without nan)              
        
        data_1[target].reset_index(drop=True, inplace=True)
        reduced_df.reset_index(drop=True, inplace=True)
        final_df = pd.concat([reduced_df,data_1[target]],axis=1)
                   
        self.data = final_df
        return final_df
    
    def split_data(self,data,target):  #Split the data
        
        print("\nspliting the data as train and test")
        X=data.drop(target,axis=1)
        y=data[target]
        from sklearn.datasets import make_classification
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,stratify =y,random_state=1)#use stratified kfold distribution for imbalanced class distributin
        return X_train, X_test, y_train, y_test
                   
    def pipe_and_filter(self):# this function is to pipe earlier steps
        new = self.treat_missing(self.data)
        new = self.drope(self.data,col= self.drop_col)
        new = self.deal_outliers(self.data,columns = self.outlier_col)
        new = self.encoder(self.data)
        new_scaled = self.scaler(self.data)
        new = self.reduce_dimension(self.data,reduction_model = self.reduction_model,dim=self.dim)
        X_train, X_test, y_train, y_test =self.split_data(self.data,self.target)
        print("DONE!")
        
        result = [X_train, X_test, y_train, y_test]
        df =[new_scaled, new]
                   
        return df,result
        
        
            
        
    

In [ ]:
new_cat_col = data.select_dtypes(exclude=['int64', 'float64']).columns #catagorical columns
corr_matrix = data.drop(new_cat_col, axis = 1).corr()
co =corr_matrix.unstack().sort_values().drop_duplicates() 

print("variance of data\n\n",data.var().sort_values())
pd.DataFrame(co,columns=['correlation']).tail(3)


- Since  emp.var.rate ,euribor3m & nr.employed are highly correlated and the variance of emp.var.rate and euribor3m is less variance  we can drop emp.var.rate and euribor3m 
- we can drop day_of week too since it has equal probable

In [30]:
outlier_col = ['age', 'campaign','cons.conf.idx']
drop_col =  ["emp.var.rate","euribor3m",'day_of_week',"duration"]


In [31]:
processor = PreProcessor(data,'y',drop_col,outlier_col,"pca",7)
new_data,split_result = processor.pipe_and_filter()


 [] variables have dropped based on missing value ratio

 4121 missing values have been dropped

 [['emp.var.rate', 'euribor3m', 'day_of_week', 'duration']] variable has been dropped based on high correlation and less variance analysis

 before dealing with the ouliers the shape of the data (41188, 16)
after removing the outliers (36967, 16)
4221  data has been removed based on outlier analysis

['housing', 'loan', 'contact', 'y'] variables have been encoded based on label encoding the rest encoded as dummy variable

---- Scaling the data based on MinMaxScaler(copy=True, feature_range=(0, 1))

The data is reduced based on pca algorithm to 8 dimentions
Executing the reduction starts and wait ......
PCA done! Time elapsed: 20.35617685317993 seconds

spliting the data as train and test
DONE!


In [32]:
#for later use
scaled_data = new_data[0]
reduced_data =new_data[1]

X_train =split_result[0]
X_test = split_result[1]
y_train =split_result[2]
y_test = split_result[3]


In [ ]:
reduced_data

# Modeling

In [34]:
from sklearn import preprocessing 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

import scipy.stats as stat
from sklearn import svm

from sklearn.metrics import *
import datetime

In [26]:
k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [68]:
class ClassifierModeling:
    def __init__(self,model_name,X_train,y_train,X_test,y_test,kfold):
        
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.kfold =kfold
        self.y_pred = None
        self.model_name = model_name
        if self.model_name== "RandomForestClassifier":
            self.model = RandomForestClassifier()
        elif self.model_name == "LogisticRegression":
            self.model = LogisticRegression(solver='liblinear', random_state=0)
        elif self.model_name == "DecisionTreeClassifier":
            self.model = DecisionTreeClassifier()
        elif self.model_name == "XG_Boost":
            data_dmatrix = xgb.DMatrix(data=self.X_train,label=self.y_train)
            self.model = xgb.XGBClassifier()
        elif self.model_name =="Multilayer Perceptron":
            print("not implemented yet")
        elif self.model_name == "svm" :
            self.model = svm.SVC(kernel='linear', C=0.01)
        elif self.model_name == "adboost":
             self.model =AdaBoostClassifier()
        elif self.model_name == "gradienBoost":
             self.model = GradientBoostingClassifier()
        
    def fit(self):
        self.model.fit(self.X_train,self.y_train)

    def get_predicate(self): 
        self.y_pred = pd.Series(self.model.predict(self.X_test),name ="predict")
        return self.y_pred 
    def get_MSE(self):
        return mean_squared_error(self.y_test,self.y_pred)
    def get_score(self):
        return -(r2_score(self.y_test,self.y_pred))
        
    def get_loss(self):
        return np.sqrt(mean_squared_error(self.y_test,self.y_pred))
    def validate_model(self):
        print
        model_fit =pd.DataFrame()
        model_fit = pd.concat([self.y_pred, self.y_test], axis=1)

        

        # matrix = confusion_matrix(self.y_test, self.y_pred)
        # d= plot_confusion_matrix(self.model, self.X_test, self.y_test,
        #                                      display_labels=["yes","no"],
        #                                      cmap=plt.cm.Blues)

        total = float(len(model_fit))
        fig, axs = plt.subplots(1,3,squeeze=False,figsize=(15, 3))
        plt.rcParams.update({'font.size': 10})
        d= plot_confusion_matrix(self.model, self.X_test, self.y_test,
                                             display_labels=["yes","no"],
                                             cmap=plt.cm.Blues,ax=axs[0,2])
        d.ax_.set_title("{} confusion matrix".format(self.model_name))
        


        for ax in axs.flatten():
            
            for i, var in enumerate(model_fit.columns):
                ax  = sns.countplot(var, data=model_fit, ax=axs[0][i])
                ax .set_title(self.model_name)
                for p in ax.patches:
                    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
                    x = p.get_x() + p.get_width()
                    y = p.get_height()
                    ax .annotate(percentage, (x, y),ha='center')
        #fig.savefig("https://github.com/muluwork-shegaw/10Academy-week6/blob/master/data/{}.png".format(self.model_name))
     
        return matrix,model_fit
    def get_eff_model(self):
        if self.model_name != "svm":
        
            metrics = pd.DataFrame()
            metrics["model"] = [self.model_name]
            metrics["MSE"] = mean_squared_error(self.y_test,self.y_pred)
            metrics["Loss"] = np.sqrt(mean_squared_error(self.y_test,self.y_pred))
            metrics["Score"] = -(r2_score(self.y_test,self.y_pred))
            metrics["Kappa"] = cohen_kappa_score(self.y_test, self.y_pred)
            metrics["ROC_Auc"] = roc_auc_score(self.y_test, self.y_pred)
            metrics["precision"] = precision_score(self.y_test, self.y_pred)
            metrics["recall"] = recall_score(self.y_test, self.y_pred)
            metrics["f1_score"] = f1_score(self.y_test, self.y_pred)
            metrics["accuracy"] = accuracy_score(self.y_test, self.y_pred)
        

        
            return metrics
    def get_accuracy_with_kfold(self):
       
        return cross_val_score(self.model,self.X_test, self.y_test,cv=self.kfold, scoring= 'accuracy').mean()
        
    def get_loss_with_kfold(self,valid_data,valid_targ,k_fold):
        return -(cross_val_score(self.model,self.X_test, self.y_test,cv=self.kfold, scoring= 'neg_log_loss').mean())
    def eff_model_with_kfold(self):
        if self.model_name != "svm":

            scoring = ["accuracy","neg_log_loss","r2",
                 "neg_mean_squared_error","neg_mean_absolute_error"] 

            metrics = pd.DataFrame()
            metrics["model"] = [self.model_name]
            for scor in scoring:
                score = []
                result = cross_val_score(estimator= self.model, X=self.X_test, y= self.y_test,cv=self.kfold,scoring=scor )
                score.append(result.mean())

                metrics[scor] =pd.Series(score)

            return metrics
        
    def get_feature_impo(self):
        if self.model_name != "LogisticRegression":
            feat_importance = pd.Series(self.model.feature_importances_, index=self.X_train.columns)
            feat_importance.plot(kind='bar')
            plt.show()
        return feat_importance
    def get_summary(self):# for feature importance of logistic regression
        if self.model_name == "LogisticRegression":

            denom = (2.0*(1.0+np.cosh(self.model.decision_function(X))))
            denom = np.tile(denom,(X.shape[1],1)).T
            F_ij = np.dot((X/denom).T,X) ## Fisher Information Matrix
            Cramer_Rao = np.linalg.inv(F_ij) ## Inverse Information Matrix
            sigma_estimates = np.sqrt(np.diagonal(Cramer_Rao))
            z_scores = self.model.coef_[0]/sigma_estimates # z-score for eaach model coefficient
            p_values = [stat.norm.sf(abs(x))*2 for x in z_scores] ### two tailed test for p-values

            z_scores = z_scores
            p_values = p_values
            sigma_estimates = sigma_estimates
            F_ij = F_ij

            summary= pd.DataFrame()
            summary["features"] = self.X_train.columns
            summary["z_score"] = self.z_scores
            summary["p_value"] = self.p_values
            sns.barplot(summary["features"],summary["p_value"], data=summary)
        return summary

    def save_model(self):
        
        now = datetime.datetime.now().strftime('%Y-%m-%d')
        # Saving model to disk
        filename = now + '.pkl'
        pickle.dump(self.model, open(filename, 'wb'))
        return filename
    def make_it_stratified(self,data,model_name,target):
        X=data.drop(target,axis=1)
        y=data[target]
        self.model_name = model_name

        # X, y = make_classification(n_samples=1000, n_classes=2, weights=[0.99, 0.01], flip_y=0, random_state=1)


        '''
        use stratified k-fold cross-validation 
        with imbalanced datasets to preserve the class distribution in the train and test 
        sets for each evaluation of a given model.
        '''
        eff_kfold =[]
        eff =[]
        model_pred = []
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
        #enumerate the splits and summarize the distributions
        for train_ix, test_ix in kfold.split(X, y):
            # select rows
            X_train, X_test = X[train_ix], X[test_ix]
            y_train, y_test = y[train_ix], y[test_ix]

            self.fit()
            self.get_predicate()
            matrix,model_fit=self.validate_model()
            model_pred.append(model_fit)
    
            eff_kfold.append(self.eff_model_with_kfold())
            eff.append(self.get_eff_model())    

            # summarize train and test composition
            train_0, train_1 = len(y_train[y_train==0]), len(y_train[y_train==1])
            test_0, test_1 = len(y_test[y_test==0]), len(y_test[y_test==1])
            print('>Train: 0=%d, 1=%d, Test: 0=%d, 1=%d' % (train_0, train_1, test_0, test_1))
        df_kfold = pd.concat(eff_kfold)
        df_eff = pd.concat(eff)
        return df_kfold,df_eff,model_pred
            
        

# Analysis

In [70]:
classifiers =["RandomForestClassifier","LogisticRegression",
              "DecisionTreeClassifier", "XG_Boost","svm","adboost","gradienBoost"]

In [ ]:
eff_kfold =[]
eff =[]
mode = []
for model_classifier in classifiers:
    classifier = ClassifierModeling(model_classifier,X_train,y_train,X_test,y_test,k_fold)
    classifier.fit()
    pridict = classifier.get_predicate()
    matrix,model_fit=classifier.validate_model()
    mode.append(model_fit)
    
    eff_kfold.append(classifier.eff_model_with_kfold())
    eff.append(classifier.get_eff_model())    


- **A confusion matrix** is a matrix (table) that can be used to measure the performance of an machine learning algorithm, usually a supervised learning one. Each row of the confusion matrix represents the instances of an actual class and each column represents the instances of a predicted class.

In [ ]:
c= y_test
d= classifier.y_pred
a = pd.concat([c,d],axis=1)

a

,y,predict
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,1.0,1.0
4,0.0,0.0
...,...,...
3692,0.0,0.0
3693,0.0,0.0
3694,0.0,0.0
3695,0.0,0.0


In [ ]:
df = pd.concat(eff)
df

,model,MSE,Loss,Score,Kappa,ROC_Auc,precision,recall,f1_score,accuracy
0,RandomForestClassifier,0.121450,0.348497,0.237006,0.202595,0.578571,0.396985,0.193627,0.260297,0.878550
0,LogisticRegression,0.110360,0.332204,0.124050,0.000000,0.500000,0.000000,0.000000,0.000000,0.889640
0,DecisionTreeClassifier,0.150663,0.388153,0.534548,0.216751,0.606165,0.308483,0.294118,0.301129,0.849337
0,XG_Boost,0.110360,0.332204,0.124050,0.003805,0.501073,0.500000,0.002451,0.004878,0.889640
0,adboost,0.110901,0.333018,0.129560,-0.001078,0.499696,0.000000,0.000000,0.000000,0.889099
0,gradienBoost,0.110360,0.332204,0.124050,0.018742,0.505367,0.500000,0.012255,0.023923,0.889640


In [ ]:
df_kfold =pd.concat(eff_kfold)
df_kfold

,model,accuracy,neg_log_loss,r2,neg_mean_squared_error,neg_mean_absolute_error
0,RandomForestClassifier,0.872600,-0.589495,-0.330594,-0.130376,-0.129564
0,LogisticRegression,0.889640,-0.321747,-0.124050,-0.110360,-0.110360
0,DecisionTreeClassifier,0.812824,-6.372992,-0.911998,-0.186362,-0.187448
0,XG_Boost,0.889371,-0.324521,-0.126763,-0.110629,-0.110629
0,adboost,0.888018,-0.667143,-0.140568,-0.111982,-0.111982
0,gradienBoost,0.888017,-0.326756,-0.140629,-0.111983,-0.111983


In [ ]:
!pip install keras

In [ ]:
!pip install tensorflow

In [ ]:
%tensorflow_version 2.2
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, Model, models, layers
import matplotlib.pyplot as plt
%matplotlib inline
from google.colab import files
import time
from tensorflow.keras.utils import Sequence
import random
import math

In [ ]:
# # demonstration of calculating metrics for a neural network model using sklearn
# from sklearn.datasets import make_circles
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import precision_score
# from sklearn.metrics import recall_score
# from sklearn.metrics import f1_score
# from sklearn.metrics import cohen_kappa_score
# from sklearn.metrics import roc_auc_score
# from sklearn.metrics import confusion_matrix
# from keras.models import Sequential
# from keras.layers import Dense

# # generate and prepare the dataset
# def get_data():

# 	return X_train, y_train, X_test, y_test

# # define and fit the model
# def get_model(trainX, trainy):
# 	# define model
# 	model = Sequential()
# 	model.add(Dense(100, input_dim=2, activation='relu'))
# 	model.add(Dense(1, activation='sigmoid'))
# 	# compile model
# 	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# 	# fit model
# 	model.fit(trainX, trainy, epochs=300, verbose=0)
# 	return model

# # generate data
# trainX, trainy, testX, testy = get_data()
# # fit model
# model = get_model(trainX, trainy)


# # predict probabilities for test set
# yhat_probs = model.predict(testX, verbose=0)
# # predict crisp classes for test set
# yhat_classes = model.predict_classes(testX, verbose=0)
# # reduce to 1d array
# yhat_probs = yhat_probs[:, 0]
# yhat_classes = yhat_classes[:, 0]

# # accuracy: (tp + tn) / (p + n)
# accuracy = accuracy_score(testy, yhat_classes)
# print('Accuracy: %f' % accuracy)
# # precision tp / (tp + fp)
# precision = precision_score(testy, yhat_classes)
# print('Precision: %f' % precision)
# # recall: tp / (tp + fn)
# recall = recall_score(testy, yhat_classes)
# print('Recall: %f' % recall)
# # f1: 2 tp / (2 tp + fp + fn)
# f1 = f1_score(testy, yhat_classes)
# print('F1 score: %f' % f1)

# # kappa
# kappa = cohen_kappa_score(testy, yhat_classes)
# print('Cohens kappa: %f' % kappa)
# # ROC AUC
# auc = roc_auc_score(testy, yhat_probs)
# print('ROC AUC: %f' % auc)
# # confusion matrix
# matrix = confusion_matrix(testy, yhat_classes)
# print(matrix)
# def eff_model(self):
#     metrics = pd.DataFrame()
#     metrics["model"] = ["Multilayer Perceptron Model"]
#     metrics["Kappa"] = cohen_kappa_score(testy, yhat_classes)
#     metrics["ROC_Auc"] = roc_auc_score(testy, yhat_probs)
#     metrics["precision"] = precision_score(testy, yhat_classes)
#     metrics["recall"] = recall_score(testy, yhat_classes)
#     metrics["f1_score"] = f1_score(testy, yhat_classes)
#     metrics["accuracy"] = accuracy_score(testy, yhat_classes)